# 🚀 Livox MID-40 Control Panel 🛰️

**Prerequisites:**

```bash
pip install ipywidgets
```

In [ ]:

import openpylivox as opl
import time
import ipywidgets as widgets
from IPython.display import display, clear_output

# Local log to widget
def log(msg):
    with out:
        print(msg)

# Sensor holder
sensor_holder = {"sensor": None}

# UI components
label = widgets.HTML("<h2>🔴 Livox MID-40 Control Panel 🛰️</h2>")
connect_btn = widgets.Button(description="🔌 Connect", button_style="primary")
start_btn = widgets.Button(description="▶️ Start Capture", button_style="success", disabled=True)
stop_btn  = widgets.Button(description="⏹️ Stop Capture", button_style="danger", disabled=True)
export_btn= widgets.Button(description="📁 Export LAS", button_style="info", disabled=True)

out = widgets.Output(layout=widgets.Layout(border='1px solid gray', padding='10px',
                                         max_height='300px', overflow='auto'))
display(label, widgets.HBox([connect_btn, start_btn, stop_btn, export_btn]), out)

# Handlers
def on_connect(b):
    out.clear_output()
    log("🔄 Connecting to Livox MID-40...")
    sensor = opl.openpylivox(False)
    sensor.showMessages(False)
    if not sensor.auto_connect():
        log("❌ Connection failed.")
        return
    sensor_holder["sensor"] = sensor
    log(f"✔️ Connected: {sensor.serialNumber()}")
    connect_btn.disabled = True
    start_btn.disabled = False

def on_start(b):
    out.clear_output()
    sensor = sensor_holder["sensor"]
    log("🔄 Spinning up LiDAR...")
    sensor.lidarSpinUp()
    log("▶️ Starting data stream...")
    sensor.dataStart_RT_B()
    # use a large duration to keep file open until manual stop
    sensor.saveDataToFile("interactive.bin", secsToWait=0.0, duration=1e6)
    log("⏳ Capturing... (click Stop Capture to end)")
    start_btn.disabled = True
    stop_btn.disabled = False

def on_stop(b):
    sensor = sensor_holder["sensor"]
    log("⏹️ Stopping stream and spinning down...")
    sensor.dataStop()
    sensor.lidarSpinDown()
    sensor.disconnect()
    log("✔️ Capture stopped.")
    stop_btn.disabled = True
    export_btn.disabled = False

def on_export(b):
    log("📦 Converting to LAS...")
    opl.convertBin2LAS("interactive.bin", deleteBin=True)
    log("✔️ interactive.bin.las created.")

connect_btn.on_click(on_connect)
start_btn.on_click(on_start)
stop_btn.on_click(on_stop)
export_btn.on_click(on_export)
